In [1]:
import numpy as np
import tensorflow as tf
import codecs
from os import listdir
import nltk
from gensim.models import Word2Vec
from random import randint
import data

In [2]:
vocab_size = 324
d, id_to_token = data.get_data(vocab_size)
for k in id_to_token:
    if id_to_token[k] == 'eos':
        id_to_token[k] = '\n'
x, y = data.sample(d, 5)
print x[:2]
print [np.argmax(j) for j in x]
print [np.argmax(j) for j in y]

324
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
dirrs = ['sentiment/train/', 'sentiment/test/']
sent = []
for dirr in dirrs:
    print dirr
    l = listdir(dirr+'pos')
    print 'pos'
    for r in l:
        t = codecs.open(dirr+'pos/'+r,'r',encoding='utf8').read()
        sent.append(nltk.word_tokenize(t))
    l = listdir(dirr+'neg')
    print 'neg'
    for r in l:
        t = codecs.open(dirr+'neg/'+r,'r',encoding='utf8').read()
        sent.append(nltk.word_tokenize(t))

In [ ]:
m = Word2Vec.load('embedding.model')
data = []
for s in sent:
    a = [m[x] for x in s]
    data.append(a)
data = np.array(data)

In [ ]:
def sam(d):
    r = randint(0, data.shape[0] - 1)
    t = d[r]
    x = t[:-1]
    y = t[1:]
    return np.array(x), np.array(y)

s = sample(data)
s[0].shape, s[1].shape

In [15]:
# hyper-parameters
epochs = 20000
hidden_layer = 50
inp_out_size = 324
learning_rate = 0.1
num_steps = 10

In [16]:
# Wxh = tf.Variable([[.1, .2], [.3, .4], [.5, .6], [.7, .8]], dtype=tf.float32)
# Whh = tf.Variable([[.1, .2], [.3, .4]], dtype=tf.float32)
# Why = tf.Variable([[.1, .2, .3, .4], [.4, .5, .6, .7]], dtype=tf.float32)
Wxh = tf.Variable(tf.random_normal([inp_out_size, hidden_layer], mean=0, stddev=0.001))
Whh = tf.Variable(tf.random_normal([hidden_layer, hidden_layer], mean=0, stddev=0.001))
Why = tf.Variable(tf.ones([hidden_layer, inp_out_size]))
bb = tf.zeros([hidden_layer], dtype=tf.float32)

In [17]:
def recurrence(prev, inp):
    i = tf.reshape(inp, shape=[1, -1])
    p = tf.reshape(prev, shape=[1, -1])
#     h = tf.nn.tanh(tf.squeeze(tf.matmul(i, Wxh)) + tf.squeeze(tf.matmul(p, Whh)) + bb)
    h = tf.tanh( tf.matmul(p, Whh) + tf.matmul(i, Wxh) + bb )
    h = tf.reshape(h, [hidden_layer])
    print h
    return h

In [18]:
a = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
b = tf.placeholder(shape=[None, inp_out_size], dtype=tf.float32)
initial = tf.placeholder(shape=[hidden_layer], dtype=tf.float32)
states = tf.scan(recurrence, a, initializer=initial)
outputs = tf.matmul(states, Why)
output_split = tf.split(0, num_steps, outputs)
target_split = tf.split(0, num_steps, b)
loss = tf.add_n([tf.nn.softmax_cross_entropy_with_logits(output_split[j], target_split[j]) for j in range(num_steps - 1, -1, -1)])
# loss = tf.sqrt(tf.reduce_sum(tf.square(tf.sub(outputs, b))))
trainables = tf.trainable_variables()
grads = tf.gradients(loss, trainables)
grads, _ = tf.clip_by_global_norm(grads, clip_norm=1.0)
grad_var_pairs = zip(grads, trainables)
optimizer = tf.train.GradientDescentOptimizer(0.01)
optimize_op = optimizer.minimize(loss)

Tensor("scan_1/while/Reshape_2:0", shape=(50,), dtype=float32)


In [19]:
def generate(sess, n):
    x, _ = data.sample(d, 1)
    gen = [id_to_token[np.argmax(x[0])]]
    for i in range(n):
        o = sess.run(outputs, {a:x, initial: np.zeros(hidden_layer)})
        o = np.argmax(o[0])
        gen.append(id_to_token[o])
        x = [0] * inp_out_size
        x[o] = 1
        x = [x]
    print gen
    print ' '.join(gen)

In [20]:
init = tf.initialize_all_variables()
sess = tf.Session()
sess.run(init)

In [22]:
print epochs
for i in range(epochs):
    x, y = data.sample(d, num_steps)
    h = np.zeros(hidden_layer)
    l, o, _ = sess.run([loss, outputs, optimize_op], {a: x, b: y, initial: h})
    if i % 3000 == 0:
        print 'epoch {0}, loss = {1}'.format(i, l)
        print [np.argmax(j) for j in o]
#             t = [id_to_token[np.argmax(j)] for j in o]
#             print t

#     x, y = data.sample(d, 10)
#     o, s = sess.run([outputs, states], {a: x, b: y})
#     print s

20000
epoch 0, loss = [ 57.80743408]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
epoch 3000, loss = [ 47.09765244]
[1, 2, 1, 1, 1, 1, 11, 3, 3, 3]
epoch 6000, loss = [ 38.35453415]
[1, 6, 4, 1, 11, 3, 3, 38, 2, 6]
epoch 9000, loss = [ 21.31944656]
[5, 3, 4, 1, 26, 3, 315, 47, 15, 155]
epoch 12000, loss = [ 17.59990501]
[13, 6, 260, 2, 1, 40, 3, 271, 3, 102]
epoch 15000, loss = [ 11.21655083]
[11, 33, 136, 3, 224, 8, 1, 11, 2, 23]
epoch 18000, loss = [ 5.99588251]
[1, 12, 3, 266, 2, 27, 101, 249, 2, 1]


In [29]:
generate(sess, 10)

['wealth', ',', '\n', 'And', 'the', 'thought', 'still', ',', '\n', 'And', 'the']
wealth , 
 And the thought still , 
 And the


In [52]:
sess.run(Whh)

array([[ -2.80843545e-02,  -2.23145187e-02,   2.55296737e-01, ...,
          8.79745930e-02,  -9.53120664e-02,   5.36855757e-02],
       [ -6.67928457e-02,   1.80839196e-01,  -3.88792939e-02, ...,
         -7.11535895e-03,   3.62812132e-02,   1.02751710e-01],
       [ -2.31132492e-01,   3.07729449e-02,  -3.23402435e-02, ...,
         -3.96190584e-03,  -4.81384844e-02,  -3.99097987e-02],
       ..., 
       [ -7.29469061e-02,   1.38710096e-01,  -4.88553494e-02, ...,
          4.21650447e-02,  -2.06389315e-02,   4.26352471e-02],
       [ -8.64108354e-02,   2.56500185e-01,   1.55067630e-02, ...,
          8.46683085e-02,   1.25194490e-01,   1.00607742e-02],
       [ -4.52823341e-02,  -6.01971708e-02,   8.82701799e-02, ...,
         -1.96572095e-02,  -1.45239681e-02,   1.67555641e-04]], dtype=float32)